In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import glob
import torch.nn.functional as F
import provider   
import argparse
import os
from tqdm import tqdm

from tensorboardX import SummaryWriter
from code.utils.cluster.IID_losses import IID_loss

In [2]:
parser = argparse.ArgumentParser(description='byol-pointnet_baseline')
# parser.add_argument('--image_folder', type=str, required = True, help='path to your folder of images for self-supervised learning')
parser.add_argument('--train_dir', default="/data2/ABC2/data_raw_clustering/pc_correct_num", required=False, help='Training data root.')
parser.add_argument('--csv_dir', default="/data2/ABC2/GT-final.csv", required=False, help='GT label data root.')
parser.add_argument('--taskType', type=str, default='Clustering', help='Type of task.')
parser.add_argument('--batchSize', type=int, default=10, help='input batch size')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=12)
parser.add_argument("--lamb", type=float, default=1.0)
args = parser.parse_args([])

In [3]:
class ABC2Dataset_pc(Dataset):
    """ABC2Dataset"""

    def __init__(self, csv_file, root_dir, task_type, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with ground truth.
            root_dir (string): Directory with all the model files.
            task_type (string): Task type.
        """

        # self.ground_truth_file = pd.read_csv(csv_file, header=0)
        self.model_id = glob.glob(os.path.join(root_dir, '*'))
        self.root_dir = root_dir
        self.task_type = task_type
        # print("model_id:", len(self.model_id))

    def __len__(self):
        return len(self.model_id)

    def __getitem__(self, idx):

        "Clustering"
        if self.task_type == "Clustering":
            model_ID = self.model_id[idx].split('/')[-1]
  
            obj_path = glob.glob(os.path.join(self.model_id[idx], '*.pt'))
#             print(obj_path)
            point_cloud_normalized = torch.load(obj_path[0])
            # print("wen_debug dataloader", point_cloud_normalized.shape)
            pc1 = point_cloud_normalized
#             point_cloud_normalized = point_cloud_normalized.unsqueeze(0)
#             # print("wen_debug dataloader", point_cloud_normalized.shape)
#             pc1 = provider.random_point_dropout(point_cloud_normalized)
#             pc1 = provider.random_scale_point_cloud(pc1)
#             pc1 = provider.shift_point_cloud(pc1)

            point_cloud_normalized = point_cloud_normalized.unsqueeze(0)
            pc2 = provider.random_point_dropout(point_cloud_normalized)
            pc2 = provider.random_scale_point_cloud(pc2)
            pc2 = provider.shift_point_cloud(pc2)

            return pc1.transpose(0,1), pc2.squeeze(0).transpose(0,1)
        
class ABC2Dataset_pc_test(Dataset):
    """ABC2Dataset"""

    def __init__(self, csv_file, root_dir, task_type, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with ground truth.
            root_dir (string): Directory with all the model files.
            task_type (string): Task type.
        """

        # self.ground_truth_file = pd.read_csv(csv_file, header=0)
        self.model_id = glob.glob(os.path.join(root_dir, '*'))
        self.root_dir = root_dir
        self.task_type = task_type
        # print("model_id:", len(self.model_id))

    def __len__(self):
        return len(self.model_id)

    def __getitem__(self, idx):

        "Clustering"
        if self.task_type == "Clustering":
            model_ID = self.model_id[idx].split('/')[-1]
  
            obj_path = glob.glob(os.path.join(self.model_id[idx], '*.pt'))
            
            point_cloud_normalized = torch.load(obj_path[0])
            # print("wen_debug dataloader", point_cloud_normalized.shape)

            return point_cloud_normalized.transpose(0,1), model_ID

In [4]:
class PointNetEncoder(nn.Module):
    def __init__(self, global_feat=True, channel=3):
        super(PointNetEncoder, self).__init__()
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, x):
        B, D, N = x.size()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        return x
        

class pointnet_cls(nn.Module):
    def __init__(self, k=2000, normal_channel=False):
        super(pointnet_cls, self).__init__()
        if normal_channel:
            channel = 6
        else:
            channel = 3
        self.feat = PointNetEncoder(global_feat=True, channel=channel)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=0.4)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x

In [5]:
train_data = ABC2Dataset_pc(args.csv_dir, args.train_dir, args.taskType)
train_loader = DataLoader(train_data, batch_size=args.batchSize, num_workers=args.workers, shuffle=True)

In [6]:
len(train_data)

22968

In [7]:
model = pointnet_cls(k=256)

opt = torch.optim.Adam(model.parameters(), lr=3e-4)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model = model.train()
writer = SummaryWriter('iic_pointnet_baseline')


In [8]:
for epoch in range(100):
    Loss = 0
    for batch_id, data in tqdm(enumerate(train_loader, 0), total=len(train_loader), smoothing=0):
        pc1, pc2 = data
        pc1, pc2 = pc1.cuda(), pc2.cuda()
        
        x_outs = model(pc1)
        x_tf_outs = model(pc2)
#         print(pc1.shape, pc2.shape, x_outs.shape, x_tf_outs.shape)
        loss, loss_no_lamb = IID_loss(x_outs, x_tf_outs, lamb=args.lamb)
#         print(loss, loss_no_lamb)
        Loss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    writer.add_scalars('train_loss', {'train_loss': Loss,}, epoch)
    print("epoch, Training loss: ", epoch, Loss)
# save your improved network
torch.save(model.state_dict(), './model_iic_k256.pt')

writer.export_scalars_to_json("./iic_pointnet_baseline.json")
writer.close()

100%|██████████| 2297/2297 [01:04<00:00, 35.57it/s]

epoch, Training loss:  0 -2563.2268825969077



100%|██████████| 2297/2297 [01:07<00:00, 34.17it/s]

epoch, Training loss:  1 -4050.952571630478



100%|██████████| 2297/2297 [01:08<00:00, 33.74it/s]

...

epoch, Training loss:  97 -6639.638449668884



100%|██████████| 2297/2297 [01:09<00:00, 33.14it/s]

epoch, Training loss:  98 -6625.514168024063



100%|██████████| 2297/2297 [01:09<00:00, 33.22it/s]

epoch, Training loss:  99 -6630.938138484955


In [9]:
class PointNetEncoder(nn.Module):
    def __init__(self, global_feat=True, channel=3):
        super(PointNetEncoder, self).__init__()
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, x):
        B, D, N = x.size()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        return x
        

class pointnet_cls(nn.Module):
    def __init__(self, k=2000, normal_channel=False):
        super(pointnet_cls, self).__init__()
        if normal_channel:
            channel = 6
        else:
            channel = 3
        self.feat = PointNetEncoder(global_feat=True, channel=channel)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=0.4)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.feat(x)
        fea = x
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return fea, x

In [11]:
train_data = ABC2Dataset_pc_test(args.csv_dir, args.train_dir, args.taskType)
train_loader = DataLoader(train_data, batch_size=args.batchSize, num_workers=args.workers, shuffle=False)

In [62]:
model = pointnet_cls(k=32)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.load_state_dict(torch.load('./model_iic_k32.pt'))
model.eval()

pointnet_cls(
  (feat): PointNetEncoder(
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=32, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)

In [63]:
modelid_list = []
fea_list = []
label_list = []
with torch.no_grad():
    for i, data in enumerate(train_loader):
        pc, id_str = data
        fea, pred = model(pc.cuda())
        label = torch.argmax(pred, 1)
        print(i, fea.shape, pc.shape,pred.shape)
        fea_list.append(fea.detach().cpu().numpy())
        modelid_list.append(id_str)
        label_list.append(label.detach().cpu().numpy())


0 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
3 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
4 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
5 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
6 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
7 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
8 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
9 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
10 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
11 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
12 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
13 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Siz

147 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
148 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
149 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
150 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
151 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
152 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
153 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
154 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
155 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
156 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
157 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
158 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
159 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
160 torch.Size([10, 1024]) torch.Size(

264 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
265 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
266 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
267 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
268 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
269 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
270 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
271 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
272 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
273 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
274 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
275 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
276 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
277 torch.Size([10, 1024]) torch.Size(

380 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
381 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
382 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
383 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
384 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
385 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
386 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
387 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
388 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
389 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
390 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
391 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
392 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
393 torch.Size([10, 1024]) torch.Size(

499 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
500 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
501 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
502 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
503 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
504 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
505 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
506 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
507 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
508 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
509 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
510 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
511 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
512 torch.Size([10, 1024]) torch.Size(

...

1787 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1788 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1789 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1790 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1791 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1792 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1793 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1794 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1795 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1796 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1797 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1798 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1799 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1800 torch.Size([10, 1024

1906 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1907 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1908 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1909 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1910 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1911 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1912 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1913 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1914 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1915 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1916 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1917 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1918 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
1919 torch.Size([10, 1024

2024 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2025 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2026 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2027 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2028 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2029 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2030 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2031 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2032 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2033 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2034 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2035 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2036 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2037 torch.Size([10, 1024

2141 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2142 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2143 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2144 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2145 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2146 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2147 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2148 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2149 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2150 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2151 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2152 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2153 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2154 torch.Size([10, 1024

2256 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2257 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2258 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2259 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2260 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2261 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2262 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2263 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2264 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2265 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2266 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2267 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2268 torch.Size([10, 1024]) torch.Size([10, 3, 4096]) torch.Size([10, 32])
2269 torch.Size([10, 1024

In [64]:
import numpy as np

In [65]:
modelid_list2 = []
for i in range(len(modelid_list)):
    modelid_list2 += modelid_list[i]

In [66]:
print(len(fea_list))
fea_list2 = np.concatenate(fea_list,0)
print(fea_list2.shape)

2297
(22968, 1024)


In [67]:
np.save("latent_space_iic_K32", fea_list2)

In [68]:
label_list2 = np.concatenate((label_list),0)

In [69]:
np.unique(label_list2)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [70]:
cluster_id_class_file = open("cluster_id_class_iic_pc_baseline_K32.txt", 'w')

for i in range(len(modelid_list2)):
    cluster_id_class_file.write(modelid_list2[i]+", "+str(label_list2[i]))
    cluster_id_class_file.write('\n')
cluster_id_class_file.close()